In [1]:
## ---- Partition Cell Set Analysis with Diff Expression Notebooks A ---- 0501.00.00
## Load Analysis Parameters (Parm1)
## Loop Through Each Partition and Perform Analysis as Follows:
## Load Partition Cell Set
## Load Partition Differential Expresssion Gene Set Model
## Plot Cell Sets as Needed, Like in 0500.00.00, but using original UMAP layout

In [2]:
## Create a Working Input and Output Data Directory, If Id Does Not Exist
parentdir <- '/gpfs/group/torkamani/devans/'
datdir <- paste(parentdir, 'CDC2', sep = '')
if (!file.exists(datdir)) {
    dir.create(datdir)
}
setwd(datdir)

In [3]:
## Read the parameters file
ps <- read.table(file = 'parms.txt', stringsAsFactors = FALSE, header = TRUE)

In [5]:
## Load Monocle3 and Seurat Libraries
library(monocle3)
library(Seurat)
library(dplyr)
library(magrittr)
library(ggplot2)
library(gridExtra)
library(Matrix)
library(rhdf5)
library(grid)

In [6]:
## Read the previously preprocessed downsampled cell set data object
down_stdycds <- readRDS(file = paste(ps$outdir,
                    'Aggregated.downsampled.QC.NoDoublets.Repartitioned.rds', sep = ''))

In [7]:
## Build a gene short name to gene id (Ensembl) lookup
short2geneid <- fData(down_stdycds)@rownames
names(short2geneid) <- fData(down_stdycds)@listData$gene_short_name

In [8]:
## Build a gene id (Ensembl) to gene short name lookup
geneid2short <- fData(down_stdycds)@listData$gene_short_name
names(geneid2short) <- fData(down_stdycds)@rownames

In [9]:
## Create variables for how cells sets are organized
cellgrps <- c('healthy', 'diseased', 'healthy', 'diseased', 'healthy', 'diseased')
cellpats <- c('ID Number 1', 'ID Number 1', 'ID Number 2', 'ID Number 2', 'ID Number 3', 'ID Number 3')

In [10]:
## Define and Assign Cell Types
celltypes6 <- c('1-Macrophages',
                '2-Endothelial Cells',
                '3-VSMCs',
                '4-Natural Killer Cells',
                '5-Cytotoxic T Lymphocytes',
                '6-B Lymphocytes')

In [11]:
## Declare Tom's best genes for definiting cell types
toms_markers5 <- c('NRXN1', 'CLU', 'ICAM2',
                 'CD14', 'CD68', 'AIF1',
                 'VWF', 'EDN1', 'ECSCR',
                 'MKI67', 'UBE2C', 'TOP2A',
                 'ACTA2', 'TAGLN', 'MYL9',
                 'ACKR1', 'SPARCL1', 'PECAM1',
                 'CALD1', 'MGP', 'DCN',
                 'NKG7', 'XCL1', 'CTSW',
                 'CD8A', 'TRAC', 'CD2',
                 'MS4A1', 'CD79A', 'BANK1',
                 'CD69', 'CXCR4', 'IL7R',
                 'LILRA4', 'IRF7', 'CLEC4C',
                 'MZB1', 'JCHAIN', 'TNFRSF17',
                 'LST1', 'FCGR3B', 'S100A8',
                 'TPSAB1', 'CPA3', 'MS4A2')
toms_gene_ids5 <- short2geneid[toms_markers5]


doug_markers1 <- c('AIF1', 'LYZ', 'FCER1G',  'CD68',
                'RNASE1', 'PECAM1', 'IGFBP4', 'ADIRF', 
                'SOD3', 'MYL9', 'CALD1', 'GSN',
                'TYROBP', 'NKG7', 'CTSW', 'CD69',
                'CD3D', 'CD2', 'TRBC2', 'TRAC',
                'MS4A1', 'CD79A', 'HLA-DQA1', 'CD37')
dougs_gene_ids1 <- short2geneid[doug_markers1]

In [47]:
## Loop through the partitions and perform analysis, plotting graphs.
## and completing differential expression analysis
np <- length(celltypes6)
gout <- TRUE
de <- TRUE
pnames <- c('macro', 'ec', 'vsmc', 'nkcells', 'tcells', 'bcells')
cellscale <- c(10, 5, 6, 30, 10, 30)
for (p in 1:6) {
    setwd(paste(datdir, '/', ps$outdir, 'partplots2', sep = ''))
    ## Extract partition subset for par
    partn_cds <- down_stdycds[, colData(down_stdycds)$assigned_cell_type == celltypes6[p]] 
    
    if (gout) {
#         ## Plot the cell set and trajectory of this partition
#         g1 <- plot_cells(partn_cds,  group_cells_by="partition",
#                show_trajectory_graph = TRUE, reduction_method = "UMAP",
#                cell_size = 0.1 * cellscale[p], alpha = .4,
#                color_cells_by = "assigned_cell_type",
#                label_cell_groups = TRUE,
#                label_leaves=TRUE,
#                label_branch_points=TRUE,
#                graph_label_size=1.5)
#         scf <- c(1, 1, 1, 1, 1, 1)
#         aspr <- c(1, 1, 1, 1, 1, 1)        
#         pdf('AssignedCellType.With.Trajectory.Rev2.pdf', width = 8 * scf[p] * aspr[p], height = 6 * scf[p])
#         print(g1)
#         dev.off()
    
#         ## Plot the new partitions of this partition, colorized by partition
#         g2 <- plot_cells(partn_cds,
#                color_cells_by="partition", group_cells_by="partition",
#                show_trajectory_graph = FALSE, reduction_method = "UMAP",
#                cell_size = 0.1 * cellscale[p], alpha = .4)
    
#         scf <- c(1, 1, 1, 1, 1, 1)
#         aspr <- c(1, 1, 1, 1, 1, 1)        
#         pdf('ColoredbyPartition.Numbered.Rev2.pdf', width = 8 * scf[p] * aspr[p], height = 6 * scf[p])
#         print(g2)
#         dev.off()
        
#         ## Plot the new partitions of this partition, colorized by partition, smaller with Legend
#         g2l <- plot_cells(partn_cds,
#                color_cells_by="partition", group_cells_by="partition",
#                show_trajectory_graph = FALSE, reduction_method = "UMAP",
#                label_cell_groups = FALSE,
#                cell_size = 0.1 * cellscale[p], alpha = .4)    
#         scf <- c(1, 1, 1, 1, 1, 1)
#         aspr <- c(1, 1, 1, 1, 1, 1)   
#         pdf('ColoredbyPartition.Numbered.WithLegned.Rev2.pdf', width = 8 * scf[p] * aspr[p], height = 6 * scf[p])
#         print(g2l)
#         dev.off()
    
        ## Plot the partitions of this partition, colorized by condition
        g3 <- plot_cells(partn_cds, color_cells_by="condition", group_cells_by="partition",
               show_trajectory_graph = FALSE, reduction_method = "UMAP",
               cell_size = 0.1 * cellscale[p], alpha = .4)
        scf <- c(1, 1, 1, 1, 1, 1)
        aspr <- c(1.2, .5, .55, .85, .9, .85)
        file1 <- paste(pnames[p], '.ColoredbyCondition.Numbered.Rev2.pdf', sep = '')
        pdf(file1, width = 8 * scf[p] * aspr[p], height = 6 * scf[p])
        print(g3)
        dev.off()
    
        ## Plot the partitions of this partition, colorized by condition, smaller with Legend
        g3l <- plot_cells(partn_cds, color_cells_by="condition", group_cells_by="partition",
               show_trajectory_graph = FALSE, reduction_method = "UMAP",
               label_cell_groups = FALSE,
               cell_size = 0.1 * cellscale[p], alpha = .4)
        scf <- c(1, 1, 1, 1, 1, 1)
        aspr <- c(1.3, .6, .67, 1, 1, 1) 
        file2 <- paste(pnames[p], '.ColoredbyCondition.Numbered.WithLegend.Rev2.pdf', sep = '')
        pdf(file2, width = 8 * scf[p] * aspr[p], height = 6 * scf[p])
        print(g3l)
        dev.off()
        

#         ## Plot the clusters of this partition, colorized by cluster
#         g4 <- plot_cells(partn_cds, color_cells_by="cluster", group_cells_by="cluster",
#                 show_trajectory_graph = FALSE, reduction_method = "UMAP",
#                 cell_size = 0.1 * cellscale[p], alpha = .4)
#         pdf('ColoredbyCluster.Numbered.Rev2.pdf', width = 8 * scf[p] * aspr[p], height = 6 * scf[p])
#         print(g4)
#         dev.off()
    }
}